In [ ]:
%load_ext tensorboard
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D

import cv2
import os

from datetime import datetime

DATADIR = "datasets"
LOGDIR = "logs/"

IMG_WIDTH = 128
IMG_HEIGHT = 128
BATCH_SIZE = 32

data_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

training_data = data_generator.flow_from_directory(
    DATADIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT)
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)


validation_data = data_generator.flow_from_directory(
    DATADIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT)
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

CONV_LAYERS = [2, 3, 4]
LAYER_SIZES = [32, 64, 128]
DENSE_LAYERS = [1, 2, 3]

for dense_layer in DENSE_LAYERS:
    for layer_size in LAYER_SIZES:
        for conv_layer in CONV_LAYERS:
            NAME = "{}  -  CONV:{} NODES: {} DENSE: {}".format(datetime.now().strftime("%Y%m%d-%H%M%S"), conv_layer, layer_size, dense_layer)
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR + NAME)
            
            model = Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=(IMG_WIDTH, IMG_HEIGHT, 1)))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for i in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            model.add(Flatten())
            
            for i in range(dense_layer):
                model.add(Dense(256))
                model.add(Activation("relu"))
            
            # Final categorisation
            model.add(Dense(3))
            model.add(Activation("softmax"))
            
            model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                         optimizer="adam",
                         metrics=["accuracy"])
            
            model.fit(training_data,
                     epochs=20,
                     steps_per_epoch=training_data.samples // BATCH_SIZE,
                     validation_data=validation_data,
                     validation_steps=validation_data.samples // BATCH_SIZE,
                     callbacks=[tensorboard_callback])
            